In [6]:
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pprint import pprint

from dataclasses import dataclass
import ase
import io

In [7]:
structures = pd.read_feather('../data/structures.feather')
train = pd.read_feather('../data/train.feather')
test = pd.read_feather('../data/test.feather')

/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [8]:
methane_name = 'dsgdb9nsd_000001'
methane = structures[structures.molecule_name == methane_name]

ethanol_name = 'dsgdb9nsd_000014'
ethanol = structures[structures.molecule_name == ethanol_name]

In [9]:
from chemistry import Bond, Atom, Molecule

In [24]:
def make_molecule(name, molecule_df):
    atoms = []
    for i, atom in molecule_df.iterrows():
        atoms.append(Atom(atom.atom, (atom.x, atom.y, atom.z)))
        
    return Molecule(name, atoms)

molecule = make_molecule('ethanol', ethanol)
pprint(molecule)

C O 2.37821202455867
O C 2.37821202455867
C O 1.4206996776198013
Name: ethanol
Atoms:
  C 0: [-0.00860504  1.50203824 -0.00681217]
  C 1: [ 0.0109931  -0.01764877 -0.01377035]
  O 2: [ 0.68088841 -0.44041803 -1.19313204]
  H 3: [ 1.01153338  1.89662027 -0.01920155]
  H 4: [-0.53159863  1.88076115 -0.88974661]
  H 5: [-0.51674569  1.87611771  0.88710737]
  H 6: [ 0.52377123 -0.38912296  0.88824081]
  H 7: [-1.02027547 -0.4050726   0.0169067 ]
  H 8: [ 0.69529575 -1.40179563 -1.20148492]
Bonds:
  C(0) - C(1)
  C(0) - H(3)
  C(0) - H(4)
  C(0) - H(5)
  C(1) - O(2)
  C(1) - H(6)
  C(1) - H(7)
  O(2) - H(8)



In [11]:
structures.head(100).molecule_name.unique()

[dsgdb9nsd_000001, dsgdb9nsd_000002, dsgdb9nsd_000003, dsgdb9nsd_000004, dsgdb9nsd_000005, ..., dsgdb9nsd_000013, dsgdb9nsd_000014, dsgdb9nsd_000015, dsgdb9nsd_000016, dsgdb9nsd_000017]
Length: 16
Categories (16, object): [dsgdb9nsd_000001, dsgdb9nsd_000002, dsgdb9nsd_000003, dsgdb9nsd_000004, ..., dsgdb9nsd_000014, dsgdb9nsd_000015, dsgdb9nsd_000016, dsgdb9nsd_000017]

In [12]:
make_molecule('', structures[structures.molecule_name == 'dsgdb9nsd_000021'])

Name: 
Atoms:
  C 0: [-0.03215888  1.54021597  0.01074456]
  C 1: [0.03381741 0.00745852 0.00180716]
  C 2: [ 0.71375573 -0.50856405 -1.27296948]
  C 3: [ 0.73848999 -0.52220887  1.25750911]
  H 4: [0.97522736 1.97414339 0.00528362]
  H 5: [-0.56196767  1.92232323 -0.8683542 ]
  H 6: [-0.54817241  1.91268408  0.90208036]
  H 7: [-0.99752527 -0.37310579  0.00977296]
  H 8: [ 1.75070024 -0.15586784 -1.33269632]
  H 9: [ 0.73446691 -1.60328293 -1.29939699]
  H 10: [ 0.19367011 -0.16085589 -2.17195511]
  H 11: [ 0.2361199  -0.18382017  2.17005253]
  H 12: [ 0.75964081 -1.61713147  1.27226102]
  H 13: [ 1.7763586  -0.16985318  1.3005302 ]
Bonds:
  C(0) - C(1)
  C(0) - H(4)
  C(0) - H(5)
  C(0) - H(6)
  C(1) - C(2)
  C(1) - C(3)
  C(1) - H(7)
  C(2) - H(8)
  C(2) - H(9)
  C(2) - H(10)
  C(3) - H(11)
  C(3) - H(12)
  C(3) - H(13)

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
from chemistry import compute_path
molecule = make_molecule('ethanol', ethanol)
compute_path(molecule, 0, 8)

[0, 1, 2, 8]